In [6]:
import os
import cv2
import numpy as np
import pickle
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as pnp
from math import ceil, log2

# === Setup ===
dataset_path = r"D:\Images_anantwave\Mulit_class\Flower Classification\Training Data"
categories = [folder for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder))]
num_images_per_category = 1000

# === SIFT Feature Extraction ===
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is None:
        return np.zeros(128)
    return descriptors.mean(axis=0)

# === Parallel Image Processing ===
def process_image(category, image_file):
    image_path = os.path.join(dataset_path, category, image_file)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is not None:
        try:
            image = cv2.resize(image, (256, 256))  # Resize to fixed size
            feat = extract_features(image)
            return feat, category
        except Exception as e:
            print(f"Failed processing {image_file}: {e}")
    return None, None

# === Feature Extraction Loop ===
all_features, labels = [], []
for category in categories:
    print(f"Processing {category} images...")
    path = os.path.join(dataset_path, category)
    valid_exts = [".jpg", ".jpeg", ".png"]
    image_files = [f for f in os.listdir(path) if os.path.splitext(f)[1].lower() in valid_exts][:num_images_per_category]

    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda f: process_image(category, f), image_files), total=len(image_files)))

    for feat, label in results:
        if feat is not None:
            all_features.append(feat)
            labels.append(label)

# === Standardization and PCA ===
scaler = StandardScaler()
scaled = scaler.fit_transform(all_features)

pca = PCA(0.95)  # retain 95% variance
reduced = pca.fit_transform(scaled)

# === Quantum Configuration ===
original_dim = reduced.shape[1]
n_qubits = int(ceil(log2(original_dim)))
dim = 2**n_qubits

def pad(vec, target_dim):
    return np.pad(vec, (0, target_dim - len(vec))) if len(vec) < target_dim else vec[:target_dim]

dev = qml.device("lightning.qubit", wires=n_qubits)

def qaoa_layer(gamma, beta, wires):
    for i in range(len(wires)):
        qml.CZ(wires=[wires[i], wires[(i+1)%len(wires)]])
    for i in range(len(wires)):
        qml.RX(gamma, wires=i)
        qml.RZ(beta, wires=i)

@qml.qnode(dev)
def quantum_model(x, gamma=0.5, beta=0.5):
    qml.AmplitudeEmbedding(x / np.linalg.norm(x), wires=range(n_qubits), normalize=True, pad_with=0.0)
    qaoa_layer(gamma, beta, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# === Quantum Encoding ===
quantum_outputs = []
print("Quantum encoding in progress...")
for vec in tqdm(reduced):
    vec_padded = pad(vec, dim)
    vec_pnp = pnp.array(vec_padded, requires_grad=False)
    result = quantum_model(vec_pnp)
    quantum_outputs.append(result)

# === Save Output ===
final_data = {
    "quantum_encoded": quantum_outputs,
    "labels": labels
}
with open("quantum_flower_data.pkl", "wb") as f:
    pickle.dump(final_data, f)

print("Quantum encoding complete for multi-class flower dataset with padded Amplitude Embedding and QAOA layer.")


Processing Aster images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:33<00:00, 29.91it/s]


Processing Daisy images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:20<00:00, 48.78it/s]


Processing Iris images...


...

Processing Sunflower images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:25<00:00, 38.63it/s]


Quantum encoding in progress...


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:26<00:00, 374.16it/s]


Quantum encoding complete for multi-class flower dataset with padded Amplitude Embedding and QAOA layer.


In [7]:
print("\n Preview of Encoded Quantum Features:")
for i in range(5):
    print(f"Label: {labels[i]}, Quantum Output: {quantum_outputs[i]}")


 Preview of Encoded Quantum Features:
Label: Aster, Quantum Output: [array(0.81240592), array(0.70811413), array(-0.05888468), array(0.21134427), array(0.37668806)]
Label: Aster, Quantum Output: [array(0.81069066), array(0.81401901), array(0.33420244), array(0.75918078), array(0.75333112)]
Label: Aster, Quantum Output: [array(0.8318538), array(0.76937374), array(0.6756186), array(0.54293519), array(0.4049052)]
Label: Aster, Quantum Output: [array(0.77895201), array(0.74673869), array(0.58030405), array(0.58846052), array(-0.35672302)]
Label: Aster, Quantum Output: [array(0.80686465), array(0.74998474), array(0.19227684), array(0.56563448), array(0.00135146)]
